In [ ]:
import random as rndm
import numpy as np # type: ignore
import time
class SudokuSolver:
    # Population size
    POPULATION = 1000
    # Number of generations
    REPETITION = 1000

    # Probability of mutation
    PM = 0.1

    # Probability of crossover
    PC = 0.95
    def __init__(self, board):
        self.board = np.array(board) # Initialize the board from a nested list input
        self.base = self.board.copy()  # Storing the original board for reference

    def find_empty(self):
         # Locate the first empty cell in the board
        for i in range(9):
            for j in range(9):
                if self.board[i][j] == 0:
                    return i, j
        return None

    def is_valid(self, num, position): 
        # Check if placing 'num' at 'position' is valid
        # Check row
        if num in self.board[position[0]]:
            return False
        # Check column
        if num in self.board[:, position[1]]:
            return False
        # Check the 3x3 box
        box_x = (position[1] // 3) * 3
        box_y = (position[0] // 3) * 3
        if num in self.board[box_y:box_y+3, box_x:box_x+3]:
            return False
        return True

    def solve_backtracking(self):
        find = self.find_empty()
        if not find:
            return True # If no empty cell is found, the puzzle is solved
        else:
            row, col = find # Get the coordinates of the empty cell

        for i in range(1, 10): # Try numbers 1 through 9 in the empty cell
            if self.is_valid(i, (row, col)): # Check if the number is valid
                self.board[row][col] = i # Place the number in the cell
                self.print_board()  # Visual step-by-step display
                print("-----")
                if self.solve_backtracking(): 
                    return True  # Recursively attempt to solve from here
                self.board[row][col] = 0 # Reset the cell if not solved
        return False # Trigger backtracking if no number fits

    def print_board(self):
         # Print the current state of the board
        for row in self.board:
            print(" ".join(str(num) for num in row))

    def is_fixed(self, pos):
      # Check if a position in the original board was filled (and hence is fixed)
      return self.base[pos[0], pos[1]] != 0
    
    def make_gene(self,initial=None): # Generate a gene for a chromosome; 'initial' is a template row
      if initial is None:
        initial = [0] * 9
      mapp = {}
      gene = list(range(1, 10))
      rndm.shuffle(gene) # Randomize the gene
        # Mapping the gene to the initial template
      for i in range(9):
        mapp[gene[i]] = i
      for i in range(9):
        if initial[i] != 0 and gene[i] != initial[i]:
            temp = gene[i], gene[mapp[initial[i]]]
            gene[mapp[initial[i]]], gene[i] = temp
            mapp[initial[i]], mapp[temp[0]] = i, mapp[initial[i]]
      return gene
    def make_chromosome(self,initial=None):
        # Create a chromosome from genes; 'initial' is a 9x9 Sudoku template
      if initial is None:
        initial = [[0] * 9] * 9
      chromosome = []
      for i in range(9):
        chromosome.append(self.make_gene(initial[i]))
      return chromosome
    
    def make_population(self,count, initial=None):
      # Generate an initial population of chromosomes
      if initial is None:
        initial = [[0] * 9] * 9
      population = []
      for _ in range(count):
        population.append(self.make_chromosome(initial))
      return population
    def get_fitness(self,chromosome):
       # Calculate the fitness of a chromosome based on Sudoku rules
      """Calculate the fitness of a chromosome."""
      fitness = 0
       # Check columns for duplicates
      for i in range(9): # For each column
        seen = {}
        for j in range(9): # Checking each cell in the column
            if chromosome[j][i] in seen:
                seen[chromosome[j][i]] += 1
            else:
                seen[chromosome[j][i]] = 1
        for key in seen: # Subtracting fitness for repeated numbers
            fitness -= (seen[key] - 1)
            # Check 3x3 boxes for duplicates
      for m in range(3): # For each 3x3 square
        for n in range(3):
            seen = {}
            for i in range(3 * n, 3 * (n + 1)):  # Checking cells in 3x3 square
                for j in range(3 * m, 3 * (m + 1)):
                    if chromosome[j][i] in seen:
                        seen[chromosome[j][i]] += 1
                    else:
                        seen[chromosome[j][i]] = 1
            for key in seen: # Subtracting fitness for repeated numbers
                fitness -= (seen[key] - 1)
      return fitness
    def crossover(self,ch1, ch2): 
      """Perform crossover between two chromosomes."""
      new_child_1 = []
      new_child_2 = []
      for i in range(9):
        x = rndm.randint(0, 1)
        if x == 1:
            new_child_1.append(ch1[i])
            new_child_2.append(ch2[i])
        elif x == 0:
            new_child_2.append(ch1[i])
            new_child_1.append(ch2[i])
      return new_child_1, new_child_2
    
    def mutation(self,ch, pm, initial):
      """Mutate a chromosome based on the mutation probability."""
      for i in range(9):
        x = rndm.randint(0, 100)
        if x < pm * 100:
            ch[i] = self.make_gene(initial[i])
      return ch
    def r_get_mating_pool(self,population):
      fitness_list = []
      pool = []
      for chromosome in population:
        fitness = self.get_fitness(chromosome)
        fitness_list.append((fitness, chromosome))
      fitness_list.sort()
      weight = list(range(1, len(fitness_list) + 1))
      for _ in range(len(population)):
        ch = rndm.choices(fitness_list, weight)[0]
        pool.append(ch[1])
      return pool
    def w_get_mating_pool(self,population):
      fitness_list = []
      pool = []
      for chromosome in population:
        fitness = self.get_fitness(chromosome)
        fitness_list.append((fitness, chromosome))
      weight = [fit[0] - fitness_list[0][0] for fit in fitness_list]
      for _ in range(len(population)):
        ch = rndm.choices(fitness_list, weights=weight)[0]
        pool.append(ch[1])
      return pool
    def get_offsprings(self,population, initial, pm, pc):
      new_pool = []
      i = 0
      while i < len(population):
        ch1 = population[i]
        ch2 = population[(i + 1) % len(population)]
        x = rndm.randint(0, 100)
        if x < pc * 100:
            ch1, ch2 = self.crossover(ch1, ch2)
        new_pool.append(self.mutation(ch1, pm, initial))
        new_pool.append(self.mutation(ch2, pm, initial))
        i += 2
      return new_pool

    def pch(self,ch):
      for i in range(9):
        for j in range(9):
            print(ch[i][j], end=" ")
        print("")
    def genetic_algorithm(self,initial_file):
        initial = initial_file
        population = self.make_population(self.POPULATION, initial)
        for _ in range(self.REPETITION):
          mating_pool = self.r_get_mating_pool(population)
          rndm.shuffle(mating_pool)
          population = self.get_offsprings(mating_pool, initial, self.PM, self.PC)
          fit = [self.get_fitness(c) for c in population]
          m = max(fit)
          if m == 0:
            return population
        return population


    def solve(self, algorithm='backtracking'): 
      
      if algorithm == 'backtracking':
        return self.solve_backtracking()
      elif algorithm == 'genetic':
        tic = time.time()
        r = self.genetic_algorithm(self.board)
        toc = time.time()
        print("time_taken: ", toc - tic)
        fit = [self.get_fitness(c) for c in r]
        m = max(fit)
        print(max(fit))

        # Printing the chromosome with the highest fitness
        for c in r:
          if self.get_fitness(c) == m:
            self.pch(c)
            break
        return True

def get_puzzle_input():
    board = []
    print("Enter your Sudoku puzzle row by row, using 0 for empty cells:")
    for i in range(9):
        while True:
            row = input(f"Row {i + 1}: ")
            if len(row) == 9 and row.isdigit():
                board.append([int(num) for num in row])
                break
            else:
                print("Invalid input. Please enter exactly 9 digits (0-9).")
    return board




def select_puzzle():
    puzzles = {
        1: [
            [5, 3, 0, 0, 7, 0, 0, 0, 0],
            [6, 0, 0, 1, 9, 5, 0, 0, 0],
            [0, 9, 8, 0, 0, 0, 0, 6, 0],
            [8, 0, 0, 0, 6, 0, 0, 0, 3],
            [4, 0, 0, 8, 0, 3, 0, 0, 1],
            [7, 0, 0, 0, 2, 0, 0, 0, 6],
            [0, 6, 0, 0, 0, 0, 2, 8, 0],
            [0, 0, 0, 4, 1, 9, 0, 0, 5],
            [0, 0, 0, 0, 8, 0, 0, 7, 9]

        ],
        2: [
            [0, 0, 3, 0, 2, 0, 6, 0, 0],
            [9, 0, 0, 3, 0, 5, 0, 0, 1],
            [0, 0, 1, 8, 0, 6, 4, 0, 0],
            [0, 0, 8, 1, 0, 2, 9, 0, 0],
            [7, 0, 0, 0, 0, 0, 0, 0, 8],
            [0, 0, 6, 7, 0, 8, 2, 0, 0],
            [0, 0, 2, 6, 0, 9, 5, 0, 0],
            [8, 0, 0, 2, 0, 3, 0, 0, 9],
            [0, 0, 5, 0, 1, 0, 3, 0, 0]
        ],
        3: [
            [2, 0, 0, 0, 8, 0, 3, 0, 0],
            [0, 6, 0, 0, 7, 0, 0, 8, 4],
            [0, 3, 0, 5, 0, 0, 2, 0, 9],
            [0, 0, 0, 1, 0, 5, 4, 0, 8],
            [0, 0, 0, 0, 0, 0, 0, 0, 0],
            [4, 0, 2, 7, 0, 6, 0, 0, 0],
            [3, 0, 1, 0, 0, 7, 0, 4, 0],
            [7, 2, 0, 0, 4, 0, 0, 6, 0],
            [0, 0, 4, 0, 1, 0, 0, 0, 3]
        ]

    }
    print("Available puzzles:")
    for key in puzzles:
        print(f"Puzzle {key}")

    choice = int(input("Select a puzzle number: "))
    return puzzles[choice]

def main():
    print("Welcome to the Sudoku Solver!")
    print("1. Enter a new puzzle")
    print("2. Choose a pre-existing puzzle")

    choice = input("Enter your choice (1 or 2): ")

    if choice == '1':
        board = get_puzzle_input()
    elif choice == '2':
        board = select_puzzle()
    else:
        print("Invalid choice")
        return

    solver = SudokuSolver(board)
    algorithm = input("Choose the algorithm (backtracking/genetic): ")
    if solver.solve(algorithm):
        if algorithm == "backtracking":
          solver.print_board()
        else:
          print("")
    else:
        print("No solution found.")

if __name__ == "__main__":
    main()

Welcome to the Sudoku Solver!
1. Enter a new puzzle
2. Choose a pre-existing puzzle
Enter your choice (1 or 2): 2
Available puzzles:
Puzzle 1
Puzzle 2
Puzzle 3
Select a puzzle number: 3
Choose the algorithm (backtracking/genetic): genetic
time_taken:  116.2238039970398
-4
2 4 5 9 8 1 3 7 6 
9 6 1 2 7 3 5 8 4 
8 3 7 5 6 4 2 1 9 
6 7 3 1 2 5 4 9 8 
1 5 9 4 3 8 6 2 7 
4 8 2 7 9 6 5 3 1 
3 9 1 6 5 7 8 4 2 
7 2 8 3 4 9 1 6 5 
5 6 4 8 1 2 7 9 3 

